# Лекция 2: Езикови модели и представяне на думи

## От думи до значение: N-грами, дистрибутивна семантика и Word2Vec

**Продължителност:** 2-2.5 часа  
**Предпоставки:** Лекция 1 (ML основи, Bag-of-Words, класификация)  
**Следваща лекция:** Токенизация (BPE, WordPiece, subword алгоритми)

---
## Цели на лекцията

След тази лекция ще можете:

- Обяснявате какво е езиков модел и защо е важен за NLP
- Изграждате N-грам модел и генерирате текст с него
- Изчислявате перплексия (perplexity) за оценка на езикови модели
- Разбирате дистрибутивната хипотеза и co-occurrence матрици
- Използвате Word2Vec за семантично сходство и аналогии

### Пътна карта за днес

```
BoW ограничения → WordNet → N-грами → Perplexity → Co-occurrence → Word2Vec → Практика
```

In [ ]:
# Основни библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re

In [ ]:
# NLP библиотеки
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize, sent_tokenize

# Сваляме необходимите данни
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✓ NLTK данните са заредени!")

In [ ]:
# Gensim за Word2Vec
import gensim.downloader as api
from gensim.models import Word2Vec

# Sklearn за PCA визуализация
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Настройки за визуализации
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

print("✓ Всички библиотеки са заредени успешно!")

---
## 1. От Bag-of-Words към значение

### Какво научихме в Лекция 1?

**Bag-of-Words (BoW)** представя текст като вектор от честоти на думи.

**Проблеми:**
- Губи реда на думите
- Няма понятие за семантично сходство
- Високомерни, разредени вектори

In [ ]:
# Припомняне: BoW губи ред и семантика
from sklearn.feature_extraction.text import CountVectorizer

sentences = [
    "king rules the kingdom",
    "queen rules the kingdom",
    "car drives on the road"
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)

print("📊 Bag-of-Words представяне:\n")
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=sentences)
print(df.to_string())

print("\n💡 Проблем: 'king' и 'queen' изглеждат напълно различни,")
print("   но 'king' и 'car' - също. Няма семантична връзка!")

### Централен въпрос: Какво означава думите да имат "значение"?

**Три подхода:**

| Подход | Идея | Пример |
|--------|------|--------|
| **Символен** | Експертни бази знания | WordNet: king IS-A ruler |
| **Дистрибутивен** | Статистика на съвместна поява | king често се среща с crown, throne |
| **Дистрибуиран** | Плътни вектори | king = [0.2, -0.5, 0.8, ...] |

---
## 2. Символни представяния: WordNet

### Какво е WordNet?

**WordNet** е лексикална база данни, организирана около **synsets** (synonym sets).

Всеки synset представлява едно **понятие** и съдържа синоними за него.

In [ ]:
# Демонстрация: Synsets за думата "dog"
print("🐕 Synsets за думата 'dog':\n")

dog_synsets = wn.synsets('dog')
for i, synset in enumerate(dog_synsets[:5]):  # Показваме първите 5
    print(f"{i+1}. {synset.name()}")
    print(f"   Дефиниция: {synset.definition()}")
    print(f"   Примери: {synset.examples()[:2]}")
    print()

### Семантични релации в WordNet

| Релация | Описание | Пример |
|---------|----------|--------|
| **Synonyms** | Думи със същото значение | {car, automobile, motorcar} |
| **Hypernyms** | По-общо понятие (IS-A) | dog → canine → mammal |
| **Hyponyms** | По-специфично понятие | dog → beagle, poodle |
| **Meronyms** | Част от цяло (HAS-A) | car has wheel, engine |

In [ ]:
# Демонстрация: Йерархия на понятия
dog = wn.synset('dog.n.01')

print(f"📊 Йерархия за '{dog.name()}':\n")

# Hypernyms (по-общи понятия)
print("⬆️ Hypernyms (по-общи):")
for hypernym in dog.hypernym_paths()[0][::-1]:  # От най-общото към dog
    print(f"   → {hypernym.name()}: {hypernym.definition()[:50]}...")

print("\n⬇️ Hyponyms (по-специфични):")
for hyponym in dog.hyponyms()[:5]:
    print(f"   → {hyponym.name()}: {hyponym.definition()[:50]}...")

In [ ]:
# Измерване на сходство в WordNet
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
car = wn.synset('car.n.01')

print("📏 Сходство между понятия (Wu-Palmer similarity):\n")

pairs = [(dog, cat), (dog, car), (cat, car)]
for s1, s2 in pairs:
    sim = s1.wup_similarity(s2)
    print(f"   {s1.name()} ↔ {s2.name()}: {sim:.3f}")

print("\n💡 dog и cat са по-сходни (и двете са животни),")
print("   докато car е от съвсем друга категория.")

### Ограничения на WordNet

- **Ръчно създаден** → скъп за поддръжка, непълен
- **Липсват нови думи:** "selfie", "bitcoin", "COVID"
- **Субективен:** граници между понятията са размити
- **Дискретни релации:** или е синоним, или не е
- **Без контекст:** "bank" = финансова или речна?

In [ ]:
# Демонстрация: WordNet не познава нови думи
new_words = ['selfie', 'bitcoin', 'emoji', 'hashtag']

print("❌ Нови думи, които WordNet не познава:\n")
for word in new_words:
    synsets = wn.synsets(word)
    status = "✓ намерен" if synsets else "✗ липсва"
    print(f"   {word}: {status}")

print("\n💡 WordNet се обновява бавно. Езикът се променя по-бързо!")

---
## 3. Статистически езикови модели: N-грами

### Какво е езиков модел?

**Езиков модел** присвоява вероятност на поредица от думи.

$$P(w_1, w_2, ..., w_n)$$

**Приложения:**
- Разпознаване на реч: "recognize speech" vs "wreck a nice beach"
- Машинен превод: кой превод е по-вероятен?
- Автокомплийт: предскажи следващата дума

### Верижно правило за вероятности

**Интуиция:** Вероятността на изречение = произведение на условни вероятности.

**Формула:**
$$P(w_1, w_2, ..., w_n) = \prod_{i=1}^{n} P(w_i | w_1, w_2, ..., w_{i-1})$$

**Проблем:** Твърде много възможни истории! Как да оценим $P(w_5 | w_1, w_2, w_3, w_4)$?

### N-грам апроксимация (Марковско допускане)

**Идея:** Приемаме, че думата зависи само от последните N-1 думи.

| Модел | Апроксимация | Пример |
|-------|--------------|--------|
| **Unigram** | $P(w_i)$ | Думата не зависи от нищо |
| **Bigram** | $P(w_i | w_{i-1})$ | Само предишната дума |
| **Trigram** | $P(w_i | w_{i-2}, w_{i-1})$ | Последните 2 думи |

In [ ]:
# Примерен корпус за демонстрация
corpus = """
I love natural language processing.
I love machine learning.
Natural language processing is fascinating.
Machine learning powers modern AI.
I study artificial intelligence.
Language models are powerful.
I love language models.
Natural language is complex.
"""

# Токенизация и preprocessing
def tokenize(text):
    """Simple tokenization with start/end tokens."""
    sentences = sent_tokenize(text.lower())
    tokenized = []
    for sent in sentences:
        tokens = ['<s>'] + word_tokenize(sent) + ['</s>']
        tokenized.append(tokens)
    return tokenized

sentences = tokenize(corpus)
print("📚 Токенизиран корпус (първите 3 изречения):\n")
for sent in sentences[:3]:
    print(f"   {sent}")

### Оценка на вероятности: Maximum Likelihood

**Формула за bigram:**
$$P(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i)}{C(w_{i-1})}$$

**Интуиция:** Колко пъти $w_i$ следва $w_{i-1}$, делено на общия брой на $w_{i-1}$.

In [ ]:
# Изграждане на bigram модел
class BigramModel:
    def __init__(self):
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
    
    def train(self, sentences):
        for sentence in sentences:
            for i in range(len(sentence) - 1):
                w1, w2 = sentence[i], sentence[i+1]
                self.bigram_counts[w1][w2] += 1
                self.unigram_counts[w1] += 1
    
    def probability(self, w2, w1):
        """P(w2 | w1)"""
        if self.unigram_counts[w1] == 0:
            return 0
        return self.bigram_counts[w1][w2] / self.unigram_counts[w1]

# Обучаваме модела
bigram_model = BigramModel()
bigram_model.train(sentences)

print("✓ Bigram модел обучен!")
print(f"   Уникални unigrams: {len(bigram_model.unigram_counts)}")
print(f"   Уникални bigram контексти: {len(bigram_model.bigram_counts)}")

In [ ]:
# Показваме вероятности след дадена дума
context = 'i'
print(f"📊 Вероятности след думата '{context}':\n")

probs = bigram_model.bigram_counts[context]
total = bigram_model.unigram_counts[context]

for word, count in probs.most_common(5):
    prob = count / total
    bar = '█' * int(prob * 20)
    print(f"   P({word:12s} | {context}) = {prob:.3f} {bar}")

In [ ]:
# Визуализация: Bigram вероятности като heatmap
# Избираме няколко интересни думи
focus_words = ['<s>', 'i', 'love', 'natural', 'language', 'machine']

# Създаваме матрица с вероятности
prob_matrix = np.zeros((len(focus_words), len(focus_words)))
for i, w1 in enumerate(focus_words):
    for j, w2 in enumerate(focus_words):
        prob_matrix[i, j] = bigram_model.probability(w2, w1)

plt.figure(figsize=(8, 6))
sns.heatmap(prob_matrix, xticklabels=focus_words, yticklabels=focus_words,
            annot=True, fmt='.2f', cmap='Blues')
plt.xlabel('Следваща дума (w₂)')
plt.ylabel('Контекст (w₁)')
plt.title('Bigram вероятности P(w₂ | w₁)')
plt.tight_layout()
plt.show()

print("💡 Ненулеви стойности показват наблюдавани bigram последователности.")

In [ ]:
# Генериране на текст с bigram модел
def generate_text(model, max_length=15):
    current = '<s>'
    result = []
    
    for _ in range(max_length):
        # Вземаме възможните следващи думи
        candidates = model.bigram_counts[current]
        if not candidates:
            break
        
        # Сумплираме пропорционално на честотите
        words = list(candidates.keys())
        counts = list(candidates.values())
        probs = np.array(counts) / sum(counts)
        
        next_word = np.random.choice(words, p=probs)
        if next_word == '</s>':
            break
        result.append(next_word)
        current = next_word
    
    return ' '.join(result)

print("🎲 Генериран текст от bigram модел:\n")
np.random.seed(42)
for i in range(5):
    text = generate_text(bigram_model)
    print(f"   {i+1}. {text}")

print("\n💡 Моделът генерира граматични фрагменти, но няма дълга кохерентност.")

### Проблеми с N-грам модели

**1. Sparsity (разреденост):**
- Много N-грами имат нулеви честоти
- $P(\text{language} | \text{artificial}) = 0$ ако не е в корпуса

**2. Storage (памет):**
- $V^N$ възможни N-грами (V = размер на речника)

**3. Без генерализация:**
- Не знае, че "dog" и "puppy" са сходни

In [ ]:
# Демонстрация: Sparsity проблем
test_bigrams = [
    ('i', 'love'),           # Виждан
    ('language', 'models'),  # Виждан
    ('artificial', 'language'),  # НЕ виждан
    ('deep', 'learning'),    # НЕ виждан
]

print("❓ Sparsity проблем: нулеви вероятности\n")
for w1, w2 in test_bigrams:
    prob = bigram_model.probability(w2, w1)
    status = "✓" if prob > 0 else "✗"
    print(f"   {status} P({w2} | {w1}) = {prob:.3f}")

print("\n💡 Невиждани bigrams имат P=0. Това е проблем!")
print("   Решение: smoothing (Laplace, Kneser-Ney, ...)")

In [ ]:
# Add-one (Laplace) smoothing
def laplace_probability(model, w2, w1, vocab_size):
    """P(w2 | w1) with Laplace smoothing."""
    count = model.bigram_counts[w1][w2]
    total = model.unigram_counts[w1]
    return (count + 1) / (total + vocab_size)

vocab_size = len(bigram_model.unigram_counts)

print("✨ С Laplace smoothing:\n")
for w1, w2 in test_bigrams:
    prob_raw = bigram_model.probability(w2, w1)
    prob_smooth = laplace_probability(bigram_model, w2, w1, vocab_size)
    print(f"   P({w2:10s} | {w1:10s}): raw={prob_raw:.3f}, smoothed={prob_smooth:.3f}")

print("\n💡 Smoothing гарантира ненулеви вероятности, но е грубо решение.")

---
## 4. Оценка на езикови модели: Perplexity

### Какво е перплексия?

**Интуиция:** Колко "изненадан" е моделът от тестовите данни?

**По-ниска перплексия = по-добър модел** (по-малко изненадан)

### Формула за перплексия

**Cross-entropy:**
$$H = -\frac{1}{N} \sum_{i=1}^{N} \log P(w_i | w_1, ..., w_{i-1})$$

**Perplexity:**
$$PP = \exp(H) = P(w_1, ..., w_N)^{-1/N}$$

**Интерпретация:** Перплексията е "ефективен branching factor" — средно колко думи са равновероятни на всяка позиция.

In [ ]:
# Изчисляване на перплексия
def compute_perplexity(model, sentences, vocab_size, use_smoothing=True):
    """Compute perplexity of bigram model on test sentences."""
    log_prob_sum = 0
    n_tokens = 0
    
    for sentence in sentences:
        for i in range(len(sentence) - 1):
            w1, w2 = sentence[i], sentence[i+1]
            
            if use_smoothing:
                prob = laplace_probability(model, w2, w1, vocab_size)
            else:
                prob = model.probability(w2, w1)
                if prob == 0:
                    prob = 1e-10  # Avoid log(0)
            
            log_prob_sum += np.log(prob)
            n_tokens += 1
    
    cross_entropy = -log_prob_sum / n_tokens
    perplexity = np.exp(cross_entropy)
    return perplexity

# Тестови изречения
test_corpus = """
I love artificial intelligence.
Machine learning is powerful.
"""
test_sentences = tokenize(test_corpus)

ppl = compute_perplexity(bigram_model, test_sentences, vocab_size)
print(f"📊 Перплексия на тестов корпус: {ppl:.2f}")

In [ ]:
# Сравнение: in-domain vs out-of-domain текст
in_domain = "I love natural language processing."
out_domain = "The cat sat on the mat."

in_domain_ppl = compute_perplexity(bigram_model, tokenize(in_domain), vocab_size)
out_domain_ppl = compute_perplexity(bigram_model, tokenize(out_domain), vocab_size)

print("📊 Сравнение на перплексия:\n")
print(f"   In-domain:  '{in_domain}'")
print(f"   Perplexity: {in_domain_ppl:.2f}\n")
print(f"   Out-domain: '{out_domain}'")
print(f"   Perplexity: {out_domain_ppl:.2f}")

print("\n💡 Моделът е по-'изненадан' от out-of-domain текст (по-висока PP).")

In [ ]:
# Визуализация: Перплексия като "branching factor"
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Лява графика: Какво означава PP
pp_values = [2, 10, 50, 100, 1000]
colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(pp_values)))

axes[0].barh(range(len(pp_values)), pp_values, color=colors)
axes[0].set_yticks(range(len(pp_values)))
axes[0].set_yticklabels([f'PP={pp}' for pp in pp_values])
axes[0].set_xlabel('Ефективен branching factor')
axes[0].set_title('Перплексия: колко думи са "равновероятни"?')

# Дясна графика: Train vs Test PP
train_ppl = compute_perplexity(bigram_model, sentences, vocab_size)
test_ppl = compute_perplexity(bigram_model, test_sentences, vocab_size)

bars = axes[1].bar(['Train corpus', 'Test corpus'], [train_ppl, test_ppl],
                   color=['steelblue', 'coral'])
axes[1].set_ylabel('Perplexity')
axes[1].set_title('Train vs Test Perplexity')

for bar, val in zip(bars, [train_ppl, test_ppl]):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{val:.1f}', ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print("💡 По-ниска перплексия = моделът по-добре предсказва данните.")

### Защо перплексията е важна?

**Универсална метрика** за езикови модели:
- N-грам модели: PP ~ 50-100
- LSTM модели: PP ~ 20-30
- GPT-2 (2019): PP ~ 17
- GPT-3 (2020): PP ~ 10-12

📌 **Ще използваме перплексия през целия курс!**

---
## 5. Дистрибутивна семантика

### Дистрибутивна хипотеза

> "You shall know a word by the company it keeps." — J.R. Firth (1957)

**Идея:** Думи, които се срещат в сходни контексти, имат сходно значение.

- "dog" и "cat" се срещат с: pet, food, loves, cute
- "car" и "truck" се срещат с: drive, road, engine, wheel

In [ ]:
# По-голям корпус за co-occurrence
bigger_corpus = """
The king rules the kingdom with wisdom.
The queen rules the kingdom with grace.
A king sits on the throne.
A queen sits on the throne.
The king wears a golden crown.
The queen wears a silver crown.
The man works in the field.
The woman works in the garden.
A man drives the car.
A woman drives the car.
The boy plays with the dog.
The girl plays with the cat.
Dogs and cats are popular pets.
The car has four wheels.
The truck has six wheels.
"""

# Токенизация
words = word_tokenize(bigger_corpus.lower())
words = [w for w in words if w.isalpha()]  # Само букви

print(f"📚 Корпус: {len(words)} думи, {len(set(words))} уникални")

### Co-occurrence матрица

**Идея:** Броим колко пъти две думи се срещат в близост (в прозорец от ±k думи).

$$M_{ij} = \text{брой пъти дума } i \text{ се среща близо до дума } j$$

In [ ]:
# Изграждане на co-occurrence матрица
def build_cooccurrence_matrix(words, window_size=2):
    """Build word-word co-occurrence matrix."""
    vocab = sorted(set(words))
    word_to_idx = {w: i for i, w in enumerate(vocab)}
    
    matrix = np.zeros((len(vocab), len(vocab)))
    
    for i, center in enumerate(words):
        # Гледаме прозорец около думата
        start = max(0, i - window_size)
        end = min(len(words), i + window_size + 1)
        
        for j in range(start, end):
            if i != j:  # Не броим думата със себе си
                context = words[j]
                matrix[word_to_idx[center], word_to_idx[context]] += 1
    
    return matrix, vocab

cooc_matrix, vocab = build_cooccurrence_matrix(words, window_size=2)
print(f"✓ Co-occurrence матрица: {cooc_matrix.shape}")
print(f"   Ненулеви елементи: {np.count_nonzero(cooc_matrix)}")

In [ ]:
# Визуализация за избрани думи
focus_words = ['king', 'queen', 'man', 'woman', 'dog', 'cat', 'car']
focus_idx = [vocab.index(w) for w in focus_words if w in vocab]
focus_words = [vocab[i] for i in focus_idx]

submatrix = cooc_matrix[np.ix_(focus_idx, focus_idx)]

plt.figure(figsize=(8, 6))
sns.heatmap(submatrix, xticklabels=focus_words, yticklabels=focus_words,
            annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Co-occurrence матрица (прозорец = 2)')
plt.tight_layout()
plt.show()

print("💡 Думи, които се срещат заедно (king-queen, dog-cat), имат високи стойности.")

### Косинусово сходство

**Идея:** Сравняваме думи по ъгъла между техните вектори.

$$\text{cos}(\vec{u}, \vec{v}) = \frac{\vec{u} \cdot \vec{v}}{||\vec{u}|| \cdot ||\vec{v}||}$$

- **1.0** = идентични посоки (много сходни)
- **0.0** = ортогонални (несвързани)
- **-1.0** = противоположни (антоними?)

In [ ]:
# Изчисляване на сходство между думи
def word_similarity(w1, w2, matrix, vocab):
    """Cosine similarity between two words."""
    i1, i2 = vocab.index(w1), vocab.index(w2)
    v1, v2 = matrix[i1], matrix[i2]
    
    dot = np.dot(v1, v2)
    norm = np.linalg.norm(v1) * np.linalg.norm(v2)
    
    if norm == 0:
        return 0
    return dot / norm

print("📏 Косинусово сходство (co-occurrence):\n")

pairs = [('king', 'queen'), ('king', 'man'), ('dog', 'cat'), ('dog', 'car')]
for w1, w2 in pairs:
    if w1 in vocab and w2 in vocab:
        sim = word_similarity(w1, w2, cooc_matrix, vocab)
        bar = '█' * int(sim * 15) if sim > 0 else ''
        print(f"   {w1:6s} ↔ {w2:6s}: {sim:.3f} {bar}")

print("\n💡 Co-occurrence улавя някои връзки, но е шумен метод.")

### Проблеми с co-occurrence вектори

- **Високомерни:** размерност = размер на речника (10K - 1M)
- **Разредени:** повечето елементи са 0
- **Неинтуитивни:** какво означават отделните измерения?

---
## 6. Word Embeddings: Word2Vec

### От разредени към плътни представяния

| Тип | Размерност | Елементи | Пример |
|-----|------------|----------|--------|
| **One-hot** | V (речник) | 0/1 | [0,0,1,0,...,0] |
| **Co-occurrence** | V | цели числа | [3,0,7,1,...,0] |
| **Word2Vec** | 50-300 | реални числа | [0.2,-0.5,0.8,...] |

### Идея на Word2Vec (Mikolov et al., 2013)

**Обучаваме невронна мрежа** да предсказва контекст от дума (или обратното).

**Две архитектури:**

| Модел | Вход | Изход | Добър за |
|-------|------|-------|----------|
| **Skip-gram** | Централна дума | Контекстни думи | Редки думи |
| **CBOW** | Контекст | Централна дума | Чести думи |

In [ ]:
# Илюстрация: Skip-gram training pairs
sentence = "the king rules the kingdom".split()
window = 2

print("📚 Skip-gram: предсказваме контекст от централна дума\n")
print(f"Изречение: {' '.join(sentence)}")
print(f"Прозорец: ±{window}\n")

print("Training pairs (center → context):")
for i, center in enumerate(sentence):
    contexts = []
    for j in range(max(0, i-window), min(len(sentence), i+window+1)):
        if j != i:
            contexts.append(sentence[j])
    
    print(f"   {center:8s} → {contexts}")

In [ ]:
# Зареждане на предтрениран Word2Vec модел
print("📥 Зареждане на Word2Vec модел (може да отнеме 1-2 минути)...\n")

# Използваме по-малък модел за бързина
word2vec = api.load('glove-wiki-gigaword-50')  # 50-мерни вектори

print(f"✓ Модел зареден!")
print(f"   Речник: {len(word2vec)} думи")
print(f"   Размерност: {word2vec.vector_size}")

In [ ]:
# Как изглежда един word vector?
word = 'king'
vector = word2vec[word]

print(f"📊 Вектор за думата '{word}':\n")
print(f"   Размерност: {len(vector)}")
print(f"   Първите 10 стойности: {vector[:10].round(3)}")
print(f"   Min: {vector.min():.3f}, Max: {vector.max():.3f}")

In [ ]:
# Намиране на сходни думи
print("🔍 Най-сходни думи:\n")

test_words = ['king', 'computer', 'happy', 'dog']
for word in test_words:
    similar = word2vec.most_similar(word, topn=5)
    neighbors = ', '.join([f"{w} ({s:.2f})" for w, s in similar])
    print(f"   {word}: {neighbors}")
    print()

### Вълшебството на Word2Vec: Аналогии

**Семантична аритметика:**
$$\vec{king} - \vec{man} + \vec{woman} \approx \vec{queen}$$

**Какво означава това?**
- Посоката от "man" към "king" = концепцията "роялти"
- Приложена върху "woman" → "queen"

In [ ]:
# Демонстрация на аналогии
print("👑 Word2Vec аналогии:\n")

analogies = [
    ('king', 'man', 'woman'),      # king - man + woman = ?
    ('paris', 'france', 'italy'),  # paris - france + italy = ?
    ('bigger', 'big', 'small'),    # bigger - big + small = ?
    ('walked', 'walk', 'swim'),    # walked - walk + swim = ?
]

for a, b, c in analogies:
    try:
        result = word2vec.most_similar(positive=[a, c], negative=[b], topn=1)
        answer = result[0][0]
        score = result[0][1]
        print(f"   {a} - {b} + {c} = {answer} (score: {score:.3f})")
    except KeyError as e:
        print(f"   Дума не е в речника: {e}")

print("\n💡 Word2Vec улавя семантични и синтактични релации!")

In [ ]:
# Визуализация на аналогия
words = ['king', 'queen', 'man', 'woman', 'prince', 'princess']
vectors = np.array([word2vec[w] for w in words])

# Редуцираме до 2D с PCA
pca = PCA(n_components=2)
coords = pca.fit_transform(vectors)

plt.figure(figsize=(10, 8))

# Точки
plt.scatter(coords[:, 0], coords[:, 1], s=100, c='steelblue', zorder=5)

# Етикети
for i, word in enumerate(words):
    plt.annotate(word, coords[i], fontsize=14, ha='center', va='bottom',
                 xytext=(0, 10), textcoords='offset points')

# Стрелки за аналогия: man→king паралелно на woman→queen
# man=2, king=0, woman=3, queen=1
plt.arrow(coords[2,0], coords[2,1], 
          coords[0,0]-coords[2,0], coords[0,1]-coords[2,1],
          head_width=0.1, head_length=0.05, fc='red', ec='red', alpha=0.7)
plt.arrow(coords[3,0], coords[3,1],
          coords[1,0]-coords[3,0], coords[1,1]-coords[3,1],
          head_width=0.1, head_length=0.05, fc='red', ec='red', alpha=0.7)

plt.title('Word2Vec: king - man + woman ≈ queen', fontsize=14)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Паралелните стрелки показват, че 'gender' е отделна посока в пространството.")

In [ ]:
# Визуализация на семантични клъстери
categories = {
    'Royalty': ['king', 'queen', 'prince', 'princess', 'throne', 'crown'],
    'Animals': ['dog', 'cat', 'horse', 'bird', 'fish', 'lion'],
    'Countries': ['france', 'germany', 'italy', 'spain', 'japan', 'china'],
    'Colors': ['red', 'blue', 'green', 'yellow', 'black', 'white']
}

# Събираме всички думи и вектори
all_words = []
all_vectors = []
all_categories = []

for cat, words in categories.items():
    for w in words:
        if w in word2vec:
            all_words.append(w)
            all_vectors.append(word2vec[w])
            all_categories.append(cat)

vectors_array = np.array(all_vectors)
coords = PCA(n_components=2).fit_transform(vectors_array)

# Цветове за категории
colors = {'Royalty': 'gold', 'Animals': 'forestgreen', 
          'Countries': 'royalblue', 'Colors': 'purple'}

plt.figure(figsize=(12, 8))
for cat in categories.keys():
    mask = [c == cat for c in all_categories]
    cat_coords = coords[mask]
    plt.scatter(cat_coords[:, 0], cat_coords[:, 1], 
                c=colors[cat], label=cat, s=100, alpha=0.7)
    
    # Етикети
    for i, (x, y) in enumerate(cat_coords):
        word = [w for w, c in zip(all_words, all_categories) if c == cat][i]
        plt.annotate(word, (x, y), fontsize=9, ha='center', va='bottom')

plt.legend(fontsize=12)
plt.title('Word2Vec: Семантични клъстери', fontsize=14)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Думи от една категория се групират заедно в embedding пространството.")

### Ограничения на Word2Vec

**1. Един вектор за дума (без полисемия):**
- "bank" (финансова) = "bank" (речна)
- Решение: контекстуални embeddings (BERT, Лекция 5)

**2. Улавя биасове от данните:**
- "doctor" е по-близо до "man", "nurse" до "woman"

**3. Статични вектори:**
- Не се променят в зависимост от контекста

In [ ]:
# Демонстрация на bias в embeddings
print("⚠️ Bias в Word Embeddings:\n")

# Проверяваме асоциации с професии
professions = ['doctor', 'nurse', 'engineer', 'teacher', 'programmer', 'secretary']

print("Сходство с 'man' vs 'woman':\n")
print(f"{'Професия':<12} {'sim(man)':<12} {'sim(woman)':<12} {'Разлика':<10}")
print("-" * 46)

for prof in professions:
    if prof in word2vec:
        sim_man = word2vec.similarity(prof, 'man')
        sim_woman = word2vec.similarity(prof, 'woman')
        diff = sim_man - sim_woman
        indicator = '→ M' if diff > 0.05 else ('→ W' if diff < -0.05 else '≈')
        print(f"{prof:<12} {sim_man:<12.3f} {sim_woman:<12.3f} {diff:+.3f} {indicator}")

print("\n💡 Embeddings улавят социални биасове от тренировъчните данни!")

---
## 7. Практически съображения

### Кога да използваме различните представяния?

| Представяне | Кога да използваме | Предимства |
|-------------|-------------------|------------|
| **Bag-of-Words** | Baseline, интерпретируемост | Просто, бързо |
| **TF-IDF** | Information retrieval, search | По-добро претегляне |
| **Word2Vec** | Семантично сходство, аналогии | Плътно, улавя семантика |
| **Контекстуални** | Полисемия, съвременни задачи | State-of-the-art |

In [ ]:
# Обучаване на собствен Word2Vec модел
custom_corpus = [
    ['machine', 'learning', 'is', 'a', 'subset', 'of', 'artificial', 'intelligence'],
    ['deep', 'learning', 'uses', 'neural', 'networks'],
    ['neural', 'networks', 'have', 'many', 'layers'],
    ['artificial', 'intelligence', 'includes', 'machine', 'learning'],
    ['nlp', 'uses', 'machine', 'learning', 'techniques'],
    ['transformers', 'revolutionized', 'nlp'],
    ['bert', 'is', 'a', 'transformer', 'model'],
    ['gpt', 'uses', 'transformer', 'architecture'],
]

# Обучаваме малък модел
custom_model = Word2Vec(
    sentences=custom_corpus,
    vector_size=20,     # Малка размерност за демо
    window=2,           # Контекстен прозорец
    min_count=1,        # Минимална честота
    epochs=100          # Брой епохи
)

print("✓ Обучен custom Word2Vec модел!")
print(f"   Речник: {len(custom_model.wv)} думи")
print(f"   Размерност: {custom_model.wv.vector_size}")

In [ ]:
# Проверяваме какво е научил
print("🔍 Какво научи custom модела?\n")

test_words = ['learning', 'neural', 'transformer']
for word in test_words:
    if word in custom_model.wv:
        similar = custom_model.wv.most_similar(word, topn=3)
        neighbors = ', '.join([f"{w} ({s:.2f})" for w, s in similar])
        print(f"   {word}: {neighbors}")

print("\n💡 На малък корпус резултатите са шумни, но улавят някои връзки.")

### Важни хиперпараметри за Word2Vec

| Параметър | Типична стойност | Ефект |
|-----------|------------------|-------|
| **vector_size** | 100-300 | По-големи = повече детайли |
| **window** | 5-10 | По-големи = по-широк контекст |
| **min_count** | 5-10 | Филтрира редки думи |
| **epochs** | 5-20 | Повече = по-добро обучение |

---
## 8. Обобщение и мост към Лекция 3

### Ключови изводи от днес

**1. Езиковите модели присвояват вероятности на текст**
- N-грам модели: просто, но sparsity проблем
- Перплексия: стандартна метрика за оценка

**2. Дистрибутивна хипотеза**
- "Думите се познават по компанията, която водят"
- Co-occurrence матрици улавят някои релации

**3. Word2Vec: плътни word embeddings**
- Улавят семантично сходство и аналогии
- Един вектор на дума → без контекст
- Съдържат биасове от данните

### Следваща лекция: Токенизация

**Въпроси, на които ще отговорим:**

- Какво е "дума"? Как делим текста?
- Как да се справим с нови думи (OOV)?
- Какво е BPE, WordPiece, SentencePiece?
- Как tokenizer-ът влияе на модела?

**Защо е важно:**
- GPT използва BPE с ~50K токени
- Токенизацията определя какво "вижда" моделът
- Критично за многоезични модели

---
## Ресурси

### Препоръчително четене

**Статии:**
1. Mikolov et al. (2013) - "Efficient Estimation of Word Representations in Vector Space"
2. Mikolov et al. (2013) - "Distributed Representations of Words and Phrases"
3. Pennington et al. (2014) - "GloVe: Global Vectors for Word Representation"

**Учебници:**
1. "Speech and Language Processing" - Jurafsky & Martin, Ch. 3 (N-grams), Ch. 6 (Embeddings)
2. "Introduction to Information Retrieval" - Manning et al., Ch. 6

**Online:**
1. Stanford CS224N Lecture 2 - Word Vectors
2. Gensim Word2Vec Tutorial
3. TensorFlow Embedding Projector

### Упражнения за вкъщи

**Упражнение 1: N-грам модел**
- Имплементирайте trigram модел
- Сравнете перплексията с bigram
- Генерирайте текст и сравнете качеството

**Упражнение 2: Word2Vec аналогии**
- Намерете 10 работещи аналогии (state:capital, verb:past_tense, ...)
- Намерете 5 примера, където аналогиите се провалят
- Защо някои работят, а други не?

**Упражнение 3: Bias анализ**
- Изследвайте bias за националности, възраст, религия
- Предложете как може да се намали bias-ът

**Упражнение 4: Сравнение на представяния**
- Имплементирайте text classification с BoW, TF-IDF, Word2Vec (средно)
- Сравнете accuracy на 20 Newsgroups dataset

---
## Край на Лекция 2

### Благодаря за вниманието!

**Въпроси?**

---

**Следваща лекция:** Токенизация (BPE, WordPiece, Subword алгоритми)